In [2]:
using JuMP
using GLPK
using GLPKMathProgInterface

# Questão 1 (Formulação)


## Variáveis de decisão

$x_{i}$ := 1 se a empresa conclui o projeto i  

$u_{i}$ := número de unidades produzidas no projeto i  


## Função objetivo

$max \sum_{i \in [5]}^{} l_{i} \cdot u_{i} - m_{i} \cdot (1 - x_{i}) - c_{i} \cdot x{i}$

## Restrições

$0 \leq u_{i} \leq x_{i} \cdot d_{i}, \forall \in [5]$  

$\sum_{i \in [5]}^{} x_{i} \leq 3$  

$\sum_{i \in [5]}^{} u_{i} \leq 10000$  

$x_{1} = x_{2}$  

$x_{5} \leq x_{4}$
   
   
$x_{i} \in \{0,1\}, \forall i : i \in [5]$  
$u_{i} \in \mathbb{Z}, \forall i : i \in [5]$

In [24]:
model = Model()
set_optimizer(model, GLPK.Optimizer);

m = [17, 8, 5, 8, 8]
c = [5, 5, 7, 7, 8]
l = [10, 11, 13, 17, 11]
d = [2, 8, 3, 1, 20]

@variable(model, x[i=1:5], Bin)
@variable(model, 0 <= u[i=1:5], Int)

@objective(model, Max, sum( (l[i]*u[i] - m[i]*(1 - x[i]) - c[i]*x[i]) for i in 1:5))

@constraint(model, con[i = 1:5], u[i] <= x[i]*d[i])
@constraint(model, sum(x[i] for i in 1:5) <= 3)
@constraint(model, sum(u[i] for i in 1:5) <= 10000)
@constraint(model, x[1]  == x[2])
@constraint(model, x[5]  <= x[4])

optimize!(model)

println("Termination Status: ", termination_status(model))
println("Objective Value: ", objective_value(model))
for i in 1:5
    println("x[$(i)] = ", value(x[i]))
    println("u[$(i)] = ", value(u[i]))
end

Termination Status: OPTIMAL
Objective Value: 229.0
x[1] = 0.0
u[1] = 0.0
x[2] = 0.0
u[2] = 0.0
x[3] = 1.0
u[3] = 3.0
x[4] = 1.0
u[4] = 1.0
x[5] = 1.0
u[5] = 20.0


# Questão 2 (Eleições)


## Variáveis de decisão

$x_{i}$ := 1 se o distrito 1 é selecionado  

$y$ := used to minimize the maximum value


## Função objetivo

$min. y$

## Restrições

$\sum_{j \in N}^{} a_{ij} \cdot x_{j} = 1, \forall i : i \in M$  

$\sum_{i \in N}^{} x_{i} = D$  

$y \geq c_{i} \cdot x_{i}, \forall i : i \in N$  

   
$x_{i} \in \{0,1\}, \forall i : i \in N$  
$y \in \mathbb{R}$

In [83]:
model = Model()
set_optimizer(model, GLPK.Optimizer);

M = 10
N = 8 # be careful with small values of N. It is necessary that at least one '1' appears in each of the N columns
D = 2

a = rand([0, 1], M, N)
c = rand(1:1000, N)

@variable(model, x[i = 1:N], Bin)
@variable(model, 0 <= y)

@objective(model, Min, y)

@constraint(model, con1[i = 1:M], sum(a[i,j]*x[j] for j in 1:N) == 1)
@constraint(model, con2, sum(x[i] for i in 1:N) == D)
@constraint(model, con3[i = 1:N], y >= c[i]*x[i])


optimize!(model)

println("Termination Status: ", termination_status(model))
println("Objective Value: ", objective_value(model))
for i in 1:N
    println("x[$(i)] = ", value(x[i]))
end

Termination Status: OPTIMAL
Objective Value: 848.0000000000006
x[1] = 1.0
x[2] = 0.0
x[3] = 0.0
x[4] = 1.0
x[5] = 0.0
x[6] = 0.0
x[7] = 0.0
x[8] = 0.0


# Questão 3 (Formulação matemática)


## Variáveis de decisão

$x_{ij}$ := 1 se busca i usa índice j

$y_{i}$ := 1 se índice i é criado

## Função objetivo

$min \sum_{i \in B}^{} \sum_{j \in I } x_{ij} \cdot c_{ij} + \sum_{i \in I} y_{i} \cdot f_{i}$

## Restrições

$x_{ij} \leq y_{j}, \forall i,j : i \in B \land j \in I$  

$\sum_{j \in I}^{} x_{ij} = 1, \forall i: i \in B$    

$\sum_{i \in I}^{} y_{i} \cdot f_{i} \leq 100$  

   
$x_{ij} \in \{0,1\}, \forall i,j : i \in B \land j in I$  
$y_{i} \in \{0,1\}, \forall i : i \in I$

In [43]:
model = Model()
set_optimizer(model, GLPK.Optimizer);

c = [[59 93 79 97 97] 
     [41 93 38 41 75] 
     [59 23 32 97 10] 
     [26 84 79 16 58] 
     [58 27 2 39 97]]
f = [86 28 3 48 25]
m = [34 21 17 56 79]

@variable(model, x[i=1:5, j=1:5], Bin)
@variable(model, y[i=1:5], Bin)

@objective(model, Min, sum(sum(x[i,j]*c[i, j] for j in 1:5) for i in 1:5 ) + sum(y[i]*f[i] for i in 1:5))

@constraint(model, con1[i = 1:5, j = 1:5], x[i, j] <= y[j])
@constraint(model, con2[i = 1:5], sum(x[i,j] for j in 1:5) == 1)
@constraint(model, sum(y[i]*m[i] for i in 1:5) <= 100)

optimize!(model)

println("Termination Status: ", termination_status(model))
println("Objective Value: ", objective_value(model))
for i in 1:5
    println("y[$(i)] = ", value(y[i]))
end
println("-------")
for i in 1:5
    for j in 1:5
        println("x[$(i), $(j)] = ", value(x[i, j]))
    end
end

Termination Status: OPTIMAL
Objective Value: 215.0
y[1] = 0.0
y[2] = 0.0
y[3] = 1.0
y[4] = 0.0
y[5] = 1.0
-------
x[1, 1] = 0.0
x[1, 2] = 0.0
x[1, 3] = 1.0
x[1, 4] = 0.0
x[1, 5] = 0.0
x[2, 1] = 0.0
x[2, 2] = 0.0
x[2, 3] = 1.0
x[2, 4] = 0.0
x[2, 5] = 0.0
x[3, 1] = 0.0
x[3, 2] = 0.0
x[3, 3] = 0.0
x[3, 4] = 0.0
x[3, 5] = 1.0
x[4, 1] = 0.0
x[4, 2] = 0.0
x[4, 3] = 0.0
x[4, 4] = 0.0
x[4, 5] = 1.0
x[5, 1] = 0.0
x[5, 2] = 0.0
x[5, 3] = 1.0
x[5, 4] = 0.0
x[5, 5] = 0.0


# Questão 4 (Formulação, Takuzu)


## Variáveis de decisão

$x_{ij}$ := 1 se a célula de coordenada (i,j) é preenchida com "1"  


## Função objetivo

$max \sum_{i,j \in [8]}^{} x_{ij}$

## Restrições

$\sum_{j \in [8]}^{} x_{ij} = 4, \forall i : i \in [8]$  

$\sum_{i \in [8]}^{} x_{ij} = 4, \forall j : j \in [8]$  

$1 \leq x_{ij} + x_{ij+1} + x_{ij+2} <= 2, \forall i,j : i \in [8] \land j \in \{1,2,3,4,5\}$  

$1 \leq x_{ij} + x_{i+1j} + x_{i+2j} <= 2, \forall i,j : j \in [8] \land i \in \{1,2,3,4,5\}$   
   
$x_{ij} \in \{0,1\}, \forall i,j : i,j \in [8]$  

In [61]:
model = Model()
set_optimizer(model, GLPK.Optimizer);

@variable(model, x[i=0:8, j=0:8], Bin)

@objective(model, Min, sum(sum(x[i,j] for j in 1:8) for i in 1:8 ))

@constraint(model, con1[i = 1:8], sum(x[i,j] for j in 1:8) == 4)
@constraint(model, con2[j = 1:8], sum(x[i,j] for i in 1:8) == 4)
@constraint(model, con3[i = 1:8, j = 1:5], 1 <= x[i, j] + x[i, j + 1] + x[i, j + 2] <= 2)
@constraint(model, con4[i = 1:5, j = 1:8], 1 <= x[i, j] + x[i+1, j] + x[i+2, j] <= 2)
@constraint(model, con5[i = 0:8], x[i, 0] == 0)
@constraint(model, con6[j = 0:8], x[0, j] == 0)


optimize!(model)

println("Termination Status: ", termination_status(model))
println("Objective Value: ", objective_value(model))
for i in 1:8
    println("$(value(x[i, 1]))  $(value(x[i, 2]))  $(value(x[i, 3]))  $(value(x[i, 4]))  $(value(x[i, 5]))  $(value(x[i, 6]))  $(value(x[i, 7]))  $(value(x[i, 8]))")
end

Termination Status: OPTIMAL
Objective Value: 32.0
1.0  0.0  0.0  1.0  0.0  0.0  1.0  1.0
0.0  1.0  0.0  1.0  0.0  1.0  1.0  0.0
0.0  0.0  1.0  0.0  1.0  1.0  0.0  1.0
1.0  1.0  0.0  1.0  1.0  0.0  0.0  0.0
0.0  0.0  1.0  1.0  0.0  1.0  1.0  0.0
0.0  1.0  1.0  0.0  1.0  0.0  0.0  1.0
1.0  1.0  0.0  0.0  1.0  0.0  1.0  0.0
1.0  0.0  1.0  0.0  0.0  1.0  0.0  1.0
